In [33]:
from tabulate import tabulate
from collections import namedtuple

class Relation():
    
    def __init__(self, list_of_tuples):
        self.tuples = set(list_of_tuples) 

    def projection(self, columns):
        '''
        Projekcija (eng. projection). 
        Metoda koja vraća sve retke zadanih stupaca.
        '''
        
        rows = list(set([tuple(getattr(row, column) 
                        for column in columns.split())
                        for row in self.tuples]))
        
        # novi namedtuple
        named = []
        new_namedtuple = namedtuple(self.name(), columns.split())
        for t in rows:
            named.append(new_namedtuple._make(t))
        return Relation(named)
        
    def selection(self, column, rel_operator, x):
        '''
        Selekcija (eng. selection).
        Metoda koja vraća samo one retke 
        koji zadovoljavaju zadani uvjet.
        '''
        
        selection = []
        if(rel_operator == "=="):
            for row in self.tuples:
                if row[column-1] == x:
                    selection.append(row)
        elif(rel_operator == "<"):
            for row in self.tuples:
                if row[column-1] < x:
                    selection.append(row)  
        elif(rel_operator == "<="):
            for row in self.tuples:
                if row[column-1] <= x:
                    selection.append(row)     
        elif(rel_operator == ">"):
            for row in self.tuples:
                if row[column-1] > x:
                    selection.append(row)
        elif(rel_operator == ">="):
            for row in self.tuples:
                if row[column-1] >= x:
                    selection.append(row)
        return Relation(selection) 

    def cartesian_product(self, r2):
        '''
        Kartezijev produkt (eng. cartesian product).
        Vraća novu relaciju koja se dobije spajajući svaki red 
        iz prve relacije sa svakim redom iz druge relacije.
        '''
        
        rows = set(x+y for x in self.tuples for y in r2.tuples)
           
        # nova relacijska shema    
        new_schema = []
        for el in self.relational_schema():
            if el in r2.relational_schema():
                new_schema.append(self.name()+"_"+el)
            else:
                new_schema.append(el)
        
        for el in r2.relational_schema():
            if el in self.relational_schema():
                new_schema.append(r2.name()+"_"+el)
            else:
                new_schema.append(el)

        # novi namedtuple
        named = []
        new_name = self.name()+"_cartesian_"+r2.name() # novo ime relacije
        new_namedtuple = namedtuple(new_name, new_schema)
        for t in rows:
            named.append(new_namedtuple._make(t))
        return Relation(named)
    
    def natural_join(self, r2):
        '''
        Prirodni spoj (eng. natural join)
        Redci rezultirajuće relacije su dobiveni na način da 
        se spajaju svi redci iz prve relacije sa svim redcima iz 
        druge relacije koji se podudaraju na zajedničkim atributima
        '''
        
        # (presjek) lista zajedničkih atributa za dvije relacije
        schema1 = self.relational_schema()
        schema2 = r2.relational_schema()
        common_fields = list(set(schema1) & set(schema2))
        
        # nova relacijska shema
        new_schema = self.relational_schema()
        for el in r2.relational_schema():
            if el not in common_fields:
                new_schema.append(el)
          
        # novo ime relacije
        new_name = self.name()+"_join_"+r2.name()
                
        new_relation = [] # nova relacija koja se dobije iz prve dvije
        
        # ako su relacijske sheme disjunktne, prirodni spoj je isto što i Kartezijev produkt
        if len(common_fields) == 0: 
            return self.cartesian_product(r2)
        else:
            indexes1 = [self.relational_schema().index(el) 
                        for el in self.relational_schema() if el in common_fields]
            indexes2 = [r2.relational_schema().index(self.relational_schema()[i]) 
                        for i in indexes1]
            
            for row1 in self.tuples:
                for row2 in r2.tuples:
                    matching = True
                    for i in range(0,len(indexes1)):
                        if row1[indexes1[i]] != row2[indexes2[i]]:
                            matching = False
                            break
                    if matching:
                        new_row = list(row1)
                        for i in range(0,len(r2.relational_schema())):
                            if i not in indexes2:
                                new_row.append(row2[i])
                        new_relation.append(tuple(new_row))
        # novi namedtuple             
        named = []
        new_namedtuple = namedtuple(new_name, new_schema)
        for t in new_relation:
            named.append(new_namedtuple._make(t))
        return Relation(named)
    


    def rename(self, current_name, new_name):
            '''
            Preimenovanje (engl. rename)
            Metoda koja omogućava promjenu imena atributa 
            u nekoj relacijskoj shemi
            '''
                             
            schema = self.relational_schema()
            
            if current_name in self.relational_schema():
                if new_name not in self.relational_schema():
                    for el in self.relational_schema():
                        if el == current_name:
                            new_index = self.relational_schema().index(el)
            
            # novi namedtuple
            named = []
            schema[new_index] = new_name
            new_namedtuple = namedtuple(self.name(), schema)
            for t in self.tuples:
                named.append(new_namedtuple._make(t))
            
            return Relation(named)
        
    def __and__(self, r2):
        '''
        Presjek (eng. intersection). 
        Metoda vraća skup svih uređenih parova 
        koji su zajednički objema relacijama.
        '''
        
        intersection = set(self.tuples) & set(r2.tuples)
                             
        # novi namedtuple             
        named = []
        new_name = self.name() + "_intersection_" + r2.name()
        new_schema = Relation(intersection).relational_schema()
        new_namedtuple = namedtuple(new_name, new_schema)
        for t in intersection:
            named.append(new_namedtuple._make(t))
        return Relation(named)

    
    def __add__(self, r2):
        '''
        Unija (eng. union). 
        Metoda vraća skup svih uređenih parova 
        koji pripadaju prvoj relaciji ili drugoj relaciji.
        '''
        
        union = set(self.tuples) | set(r2.tuples)
        return union
    
    def __or__(self, r2):
        '''
        Unija (eng. union). 
        Metoda koja poziva metodu __add__(self, r2)
        '''
        union = self + r2
        
        # novi namedtuple  
        named = []
        new_name = self.name() + "_union_" + r2.name()
        new_schema = Relation(union).relational_schema()
        new_namedtuple = namedtuple(new_name, new_schema)
        for t in union:
            named.append(new_namedtuple._make(t))
        return Relation(named)
    
    def __sub__(self, r2):
        '''
        Razlika (eng. difference). 
        Vraća skup svih uređenih parova koji pripadaju 
        prvoj relaciji, ali ne pripadaju drugoj relaciji.
        '''
        
        difference = set(self.tuples) - set(r2.tuples)
        
        # novi namedtuple  
        named = []
        new_name = self.name() + "_difference_" + r2.name()
        new_schema = Relation(difference).relational_schema()
        new_namedtuple = namedtuple(new_name, new_schema)
        for t in difference:
            named.append(new_namedtuple._make(t))
        return Relation(named)
        
    def relational_schema(self):
        columns = ""
        for column in list(self.tuples)[0]._fields:
            columns += column + " "
        columns = columns.split()
        return columns
    
    def name(self):
        return str(list(self.tuples)[0]).split('(')[0]  

    def table(self):
        print(self.name()+"\n")
        t = self.tuples
        s = self.relational_schema()
        print(tabulate(t, s, tablefmt="github"))
        
    def __eq__(self, r2):
        tuples = self.tuples == r2.tuples
        schema = self.relational_schema() == r2.relational_schema()
        name = self.name() == r2.name()
        return tuples and schema and name
        
    def __repr__(self):
        return f"Relation({self.tuples})"
        

In [34]:
import unittest
from collections import namedtuple

Osoba = namedtuple('Osoba', 'JMBAG ime spol godine')
Smjer = namedtuple('Smjer', 'JMBAG ime smjer')

R1 = namedtuple('R1', 'A B')
S1 = namedtuple('S1', 'C D E')

R2 = namedtuple('R2', 'A B C D')
S2 = namedtuple('S2', 'B C E')

N1 = namedtuple('r', 'A B C')
N2 = namedtuple('s', 'A B C')
N3 = namedtuple('k', 'A B C')

class TestRelation(unittest.TestCase):    
    def setUp(self):    
        studenti = set([
        Osoba('0178982471', 'Ana', 'žensko', 19),
        Osoba('0137829897', 'Luka', 'muško', 19),
        Osoba('0198274011', 'Marko', 'muško', 20),
        Osoba('0066719843','Mate', 'muško', 23),
        Osoba('0678124712', 'Sabina', 'žensko', 23),
        Osoba('0177023974', 'Sabina', 'žensko', 22),
        ])
        
        self.rel1 = Relation(studenti)
        
        smjerovi = set([
        Smjer('0172885615', 'Antonija', 'Informacijsko i programsko inženjerstvo'),
        Smjer('0178982471', 'Ana', 'Informacijsko i programsko inženjerstvo'),
        Smjer('0137829897', 'Luka', 'Organizacija poslovnih sustava'),
        Smjer('0198274011', 'Marko', 'Baze podataka i baze znanja'),
        Smjer('0066719843', 'Mate', 'Informatika u obrazovanju'),
        Smjer('0678124712', 'Sabina', 'Baze podataka i baze znanja'),
        Smjer('0177023974', 'Sabina', 'Organizacija poslovnih sustava'),
        ])
        
        self.rel2 = Relation(smjerovi)
        
        r1 = set([R1(4,1), R1(3,2),])
        self.rel3 = Relation(r1)
        
        s1 = set([S1(3,2,1), S1(3,3,5),])
        self.rel4 = Relation(s1)
        
        r2 = set([R2('a', 'b','c','d'), R2('a','b','a','a'),])
        self.rel5 = Relation(r2)
        
        s2 = set([S2('b','c','a'), S2('c','c','a'),])
        self.rel6 = Relation(s2)
        
        
        r1 = set([N1(2,3,1), N1(1,1,1),N1(0,1,0),])
        self.r = Relation(r1)
        
        s1 = set([N2(1,0,1), N2(2,3,1),])
        self.s = Relation(s1)
        
        k1 = set([N3(0,1,0), N3(1,2,3),])
        self.k = Relation(k1)
        
    
    def test_selection_1(self):
        a = self.rel1.selection(2, "==", "Sabina")
        b = Relation({Osoba(JMBAG='0678124712', ime='Sabina', spol='žensko', godine=23), 
                      Osoba(JMBAG='0177023974', ime='Sabina', spol='žensko', godine=22)})
        self.assertEqual(a,b)
        
    def test_selection_2(self):
        a = self.rel1.selection(1, "==", "0178982471")
        b = Relation({Osoba(JMBAG='0178982471', ime='Ana', spol='žensko', godine=19)})
        self.assertEqual(a,b)
        
    def test_selection_3(self):
        a = self.rel1.selection(4, "==", 23)
        b = Relation({Osoba(JMBAG='0066719843', ime='Mate', spol='muško', godine=23), 
                      Osoba(JMBAG='0678124712', ime='Sabina', spol='žensko', godine=23)})
        self.assertEqual(a,b)   
        
    def test_selection_4(self):
        a = self.rel1.selection(4, "<", 20)
        b = Relation({Osoba(JMBAG='0137829897', ime='Luka', spol='muško', godine=19), 
                      Osoba(JMBAG='0178982471', ime='Ana', spol='žensko', godine=19)})
        self.assertEqual(a,b)
        
    def test_selection_5(self):
        a = self.rel1.selection(4, "<=", 20)
        b = Relation({Osoba(JMBAG='0198274011', ime='Marko', spol='muško', godine=20), 
                      Osoba(JMBAG='0178982471', ime='Ana', spol='žensko', godine=19), 
                      Osoba(JMBAG='0137829897', ime='Luka', spol='muško', godine=19)})
        self.assertEqual(a,b)
        
    def test_selection_6(self):
        a = self.rel1.selection(4, ">", 20)
        b = Relation({Osoba(JMBAG='0177023974', ime='Sabina', spol='žensko', godine=22), 
                      Osoba(JMBAG='0066719843', ime='Mate', spol='muško', godine=23), 
                      Osoba(JMBAG='0678124712', ime='Sabina', spol='žensko', godine=23)})
        self.assertEqual(a,b)
    
    def test_selection_7(self):
        a = self.rel1.selection(4, ">=", 20) 
        b = Relation({Osoba(JMBAG='0177023974', ime='Sabina', spol='žensko', godine=22), 
                      Osoba(JMBAG='0198274011', ime='Marko', spol='muško', godine=20), 
                      Osoba(JMBAG='0066719843', ime='Mate', spol='muško', godine=23), 
                      Osoba(JMBAG='0678124712', ime='Sabina', spol='žensko', godine=23)})
        self.assertEqual(a,b)
        
    def test_projection_1(self):
        a = self.rel1.projection('ime godine').tuples
        a_tuples = [tuple(t) for t in a]
        b = [('Sabina', 23),('Sabina', 22), ('Ana', 19),('Luka', 19), ('Marko', 20),('Mate', 23)]
        self.assertEqual(set(a_tuples), set(b))
        
    def test_projection_2(self):
        a = self.rel1.projection('ime').tuples
        a_tuples = [tuple(t) for t in a]
        b = [('Sabina',), ('Luka',), ('Marko',), ('Mate',), ('Ana',)]
        self.assertEqual(set(a_tuples), set(b))
        
    def test_rename(self):
        a = self.rel1.rename("ime", "Smjer_ime")
        s = a.relational_schema()
        b = ['JMBAG', 'Smjer_ime', 'spol', 'godine']
        self.assertEqual(s, b)
        
    def test_natural_join1(self):
        a = self.rel1.natural_join(self.rel2).tuples
        a_tuples = [tuple(t) for t in a]
        b = [('0066719843', 'Mate', 'muško', 23, 'Informatika u obrazovanju'),
             ('0198274011', 'Marko', 'muško', 20, 'Baze podataka i baze znanja'),
             ('0177023974', 'Sabina', 'žensko', 22, 'Organizacija poslovnih sustava'),
             ('0137829897', 'Luka', 'muško', 19, 'Organizacija poslovnih sustava'),
             ('0178982471','Ana','žensko', 19, 'Informacijsko i programsko inženjerstvo'),
             ('0678124712', 'Sabina', 'žensko', 23, 'Baze podataka i baze znanja')]
        self.assertEqual(set(a_tuples),set(b))
        
    def test_natural_join2(self):
        # za disjunktne sheme
        
        a = self.rel3.natural_join(self.rel4).tuples
        a_tuples = [tuple(t) for t in a]
        b = [(3, 2, 3, 2, 1), (3, 2, 3, 3, 5), (4, 1, 3, 2, 1), (4, 1, 3, 3, 5)]
        self.assertEqual(set(a_tuples),set(b))
        
    def test_cartesian_product1(self):
        a = self.rel3.cartesian_product(self.rel4).tuples
        a_tuples = [tuple(t) for t in a]
        b = [(3, 2, 3, 2, 1), (3, 2, 3, 3, 5), (4, 1, 3, 2, 1), (4, 1, 3, 3, 5)]
        self.assertEqual(set(a_tuples),set(b))
        
    def test_cartesian_product2(self):    
        a = self.rel5.cartesian_product(self.rel6).tuples
        a_tuples = [tuple(t) for t in a]
        b = [('a', 'b', 'c', 'd', 'c', 'c', 'a'),
             ('a', 'b', 'a', 'a', 'b', 'c', 'a'),
             ('a', 'b', 'a', 'a', 'c', 'c', 'a'),
             ('a', 'b', 'c', 'd', 'b', 'c', 'a')]
        self.assertEqual(set(a_tuples),set(b))
    
    def test_cartesian_product_schema2(self):
        a = (self.rel5.cartesian_product(self.rel6)).relational_schema()
        b = ['A', 'R2_B', 'R2_C', 'D', 'S2_B', 'S2_C', 'E']
        
        self.assertEqual(a,b)
        
    def test_intersection(self):
        a = (self.r | self.s).tuples
        a_tuples = [tuple(t) for t in a]
        b = [(1, 1, 1), (0, 1, 0), (2, 3, 1), (1, 0, 1)]
        self.assertEqual(set(a_tuples), set(b))
        
    def test_union(self):
        a = (self.r & self.s).tuples
        a_tuples = [tuple(t) for t in a]
        b = [(2, 3, 1)]
        self.assertEqual(set(a_tuples), set(b))
        
    def test_difference(self):
        a = (self.r - self.s).tuples
        a_tuples = [tuple(t) for t in a]
        b = [(1, 1, 1), (0, 1, 0)]
        self.assertEqual(set(a_tuples), set(b))

    
unittest.main(argv=[''], verbosity=2, exit=False)

test_cartesian_product1 (__main__.TestRelation) ... ok
test_cartesian_product2 (__main__.TestRelation) ... ok
test_cartesian_product_schema2 (__main__.TestRelation) ... ok
test_difference (__main__.TestRelation) ... ok
test_intersection (__main__.TestRelation) ... ok
test_natural_join1 (__main__.TestRelation) ... ok
test_natural_join2 (__main__.TestRelation) ... ok
test_projection_1 (__main__.TestRelation) ... ok
test_projection_2 (__main__.TestRelation) ... ok
test_rename (__main__.TestRelation) ... ok
test_selection_1 (__main__.TestRelation) ... ok
test_selection_2 (__main__.TestRelation) ... ok
test_selection_3 (__main__.TestRelation) ... ok
test_selection_4 (__main__.TestRelation) ... ok
test_selection_5 (__main__.TestRelation) ... ok
test_selection_6 (__main__.TestRelation) ... ok
test_selection_7 (__main__.TestRelation) ... ok
test_union (__main__.TestRelation) ... ok

----------------------------------------------------------------------
Ran 18 tests in 0.031s

OK
